# Import

In [26]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn import datasets
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve
from scipy.stats import randint as sp_randint


# Data Load

In [27]:
arrest_data = pd.read_csv("arrests_w_census_loc.csv")
arrest_data.PERP_RACE[arrest_data.PERP_RACE.str.contains("WHITE")]=0
arrest_data.PERP_RACE[arrest_data.PERP_RACE != 0]=1
arrest_data = arrest_data.groupby(["PERP_RACE","BlockLocation"]).size().reset_index(name='counts')
blockLocation = arrest_data["BlockLocation"]
blockLat = [float(re.findall(r'[-\d\.]+', bl)[0]) for bl in blockLocation]
blockLon = [float(re.findall(r'[-\d\.]+', bl)[1]) for bl in blockLocation]
arrest_data["blockLat"]=blockLat
arrest_data["blockLon"]=blockLon
arrest_data = arrest_data.drop("BlockLocation", axis=1)
arrest_data = arrest_data.rename(columns={"counts": "Num_Arrests", "PERP_RACE": "Race"})
block_data = pd.read_csv("census_block_loc.csv")
block_data = pd.merge(left=arrest_data, right=block_data,
                      left_on=["blockLat","blockLon"], right_on=["Latitude","Longitude"])
census_data = pd.read_csv("nyc_census_tracts.csv")
tracts = block_data["BlockCode"]
tracts = [int(str(tract)[:-4]) for tract in tracts]
block_data["tracts"]=tracts
block_data = block_data.drop(columns=["Latitude","Longitude","BlockCode","County","blockLat","blockLon"])
data = pd.merge(left=block_data, right=census_data, left_on="tracts", right_on="CensusTract")
data = data.drop("tracts", axis=1)
data.head(n=7)

/home/erik7brown/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/erik7brown/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Race,Num_Arrests,State,CensusTract,County,Borough,TotalPop,Men,Women,Hispanic,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,0,1,NY,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
1,0,4,NY,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
2,0,1,NY,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
3,0,1,NY,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
4,0,2,NY,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
5,1,1,NY,36085024402,Richmond,Staten Island,4241,2023,2218,3.7,...,1.1,0.6,4.0,44.3,2046,75.2,21.2,3.6,0.0,8.3
6,0,3,NY,36085024401,Richmond,Staten Island,6408,2755,3653,8.3,...,0.1,0.0,3.9,44.6,2703,70.2,29.3,0.6,0.0,6.6


In [28]:
data.shape

(10955, 39)

In [29]:
data_train = data.sample(frac=0.7,random_state=200) #meant to randomly grab 70% of data by row for test
data_test  = data.drop(data_train.index)

In [30]:
cols = data.shape[1] #num columns

X = data_train.iloc[:,0:cols-1] # iloc slicing function , split the data into X and Y
y = data_train.iloc[:,cols-1:cols]


In [31]:
X.columns

Index(['Race', 'Num_Arrests', 'State', 'CensusTract', 'County', 'Borough',
       'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native',
       'Asian', 'Citizen', 'Income', 'IncomeErr', 'IncomePerCap',
       'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Professional', 'Service',
       'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit',
       'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed',
       'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork'],
      dtype='object')

In [32]:
def model_fit(my_kernel,X,y):
    clf = SVR(kernel = my_kernel, gamma='auto')
    clf.fit(X,y)
    print('================'+str(my_kernel)+'==================')
    cross_val(clf, X, y)

In [33]:
def cross_val(clf, X, y ):
    CV = KFold(n_splits=50, random_state=None, shuffle=False)
    y_pred = cross_val_predict(clf, X, y, cv = CV)
    print('Classification Report: \n')
    print(metrics.classification_report(y, y_pred, sample_weight = y,labels=np.unique(y_pred)))
    print('\nConfusion Matrix: ')
    print(metrics.confusion_matrix(y, y_pred, sample_weight = y))

# DONT RUN THIS ONE UNTIL READY

In [23]:
kernals = ['linear', 'poly', 'rbf', 'sigmoid']
CV = 10
for kernal in kernals:
    model_fit(kernal,X,y)

ValueError: could not convert string to float: 'Manhattan'

In [36]:
X['Non-White'] = 100 - X['White']

In [37]:
X = X.drop(columns=['Hispanic','Black','Native','Asian'])